In [21]:
import pretty_midi
import numpy as np
import pickle

num_notes = 128 # Number of pitches in MIDI
steps_per_sec = 24

In [52]:
data = np.load('test_history.npy', allow_pickle=True)
inst = np.load('test_instruments.npy', allow_pickle=True)

In [55]:
def data_to_midi(seq, instrument_number):
    instrument=pretty_midi.Instrument(program=instrument_number,name=pretty_midi.program_to_instrument_name(instrument_number))
    
    # Keys are pitches, values are (velocity, start time)
    notes_on = dict()
    
    for tstep in range(seq.shape[0]):
        # Check if any notes are being turned on or off
        actions = np.flip(np.sort(np.argwhere(seq[tstep, :2*num_notes])))
        for action in actions:
            if num_notes <= action < 2*num_notes:
                # Handle note-off events
                pitch = int(action - num_notes)
                if pitch in notes_on:
                    velocity = notes_on[pitch][0]
                    start_time = notes_on[pitch][1]
                    end_time = tstep/steps_per_sec
                    note=pretty_midi.Note(velocity=velocity, pitch=pitch, start=start_time, end=end_time)
                    instrument.notes.append(note)
                    del notes_on[pitch]
            else:
                # Handle note-on events
                pitch = int(action)
                velocity = int(round(seq[tstep, 2*num_notes + pitch]*127))
                start_time = tstep/steps_per_sec
                notes_on[pitch] = (velocity, start_time)
    return instrument
    

In [56]:
recording = pretty_midi.PrettyMIDI()
for i,j in zip(data, inst):
    recording.instruments.append(data_to_midi(i,j))

[45]


In [47]:
recording.write("test_midi.mid")

In [26]:
file = open( "preprocessed_data_sync_fnames.p", "rb" )
res = pickle.load(file)
file.close()

In [27]:
res[0]

'Faure/2167_gr_f45m2.mid'